## Use pandas to create dataframe from duckdb data warehouse

In [1]:
import pandas as pd
import duckdb
# ads_data_warehouse.duckdb - mart.mart_job_ads , mart.mart_hälso_sjukvård , mart.mart_pedagogik , mart.mart_teknisk_inriktning 
with duckdb.connect("../ads_data_warehouse.duckdb") as con:
    df = con.execute("SELECT * FROM mart.mart_job_ads").df()
    df_healthcare = con.execute("SELECT * FROM mart.mart_hälso_sjukvård").df()
    df_education = con.execute("SELECT * FROM mart.mart_pedagogik").df()
    df_technical = con.execute("SELECT * FROM mart.mart_teknisk_inriktning").df()

df_technical["occupation_field"].head()
df_healthcare["occupation_field"].head()
df_education["occupation_field"].head()
df["occupation_field"].unique()

df["workplace_region"].value_counts().head(20)
df["workplace_city"].value_counts()
df["workplace_municipality"].value_counts().head(20)

workplace_municipality
Göteborg        4241
Malmö           2781
Stockholm       2468
Linköping       2275
Uppsala         2085
Örebro          2050
Lund            1385
Södertälje       919
Borås            709
Västerås         557
Ej Angiven       539
Jönköping        518
Sundsvall        493
Kungsbacka       425
Östersund        404
Helsingborg      389
Ludvika          364
Örnsköldsvik     362
Halmstad         334
Finspång         326
Name: count, dtype: int64

In [2]:
df_technical_occupation = df_technical.groupby("occupation")["vacancies"].sum().reset_index(name="total vacancies").sort_values("total vacancies", ascending=False).reset_index(drop=True)
df_education_occupation = df_education.groupby("occupation")["vacancies"].sum().reset_index(name="total vacancies").sort_values("total vacancies", ascending=False).reset_index(drop=True)
df_healthcare_occupation = df_healthcare.groupby("occupation")["vacancies"].sum().reset_index(name="total vacancies").sort_values("total vacancies", ascending=False).reset_index(drop=True)

df_healthcare_occupation.head()
df_technical_occupation.head()
df_education_occupation.head()


,occupation,total vacancies
0,Elevassistent,4879
1,"Ämneslärare, 7-9",4489
2,"Grundlärare, 4-6",3878
3,"Grundlärare, fritidshem",2299
4,Lärarassistent,1516


In [3]:
df.columns

Index(['occupation_id', 'job_details_id', 'employer_id',
       'auxillary_attributes_id', 'vacancies', 'relevance',
       'application_deadline', 'occupation_id_1', 'occupation',
       'occupation_group', 'occupation_field', 'job_details_id_1', 'headline',
       'description', 'description_html', 'duration', 'salary_type',
       'salary_description', 'working_hours_type', 'scope_of_work_min',
       'scope_of_work_max', 'employer_id_1', 'employer_name',
       'employer_workplace', 'employer_organization_number',
       'workplace_street_address', 'workplace_region', 'workplace_postcode',
       'workplace_municipality', 'workplace_city', 'workplace_country',
       'auxillary_attributes_id_1', 'experience_required', 'driver_licence',
       'access_to_own_car'],
      dtype='object')

In [4]:
def group_by_city(df):
    return df.groupby("workplace_municipality")["vacancies"].sum().reset_index(name="total vacancies").sort_values("total vacancies", ascending=False).reset_index(drop=True)

def group_by_occupation(df):
    return df.groupby("occupation")["vacancies"].sum().reset_index(name="total vacancies").sort_values("total vacancies", ascending=False).reset_index(drop=True)

In [10]:
# df_technical_city = df_technical.groupby("workplace_city")["vacancies"].sum().reset_index(name="total vacancies").sort_values("total vacancies", ascending=False).reset_index(drop=True)
# df_pedagogik_city = df_education.groupby("workplace_city")["vacancies"].sum().reset_index(name="total vacancies").sort_values("total vacancies", ascending=False).reset_index(drop=True)
# df_healthcare_city = df_healthcare.groupby("workplace_city")["vacancies"].sum().reset_index(name="total vacancies").sort_values("total vacancies", ascending=False).reset_index(drop=True)
df_technical_city = group_by_city(df_technical)
df_education_city = group_by_city(df_education)
df_healthcare_city = group_by_city(df_healthcare)
df_city = group_by_city(df)

df_technical_occupation = group_by_occupation(df_technical)
df_education_occupation = group_by_occupation(df_education)
df_healthcare_occupation = group_by_occupation(df_healthcare)
df_city.head()

# gör funktion för att groupa en kolumn med vacancies, så man kan skicka in en df och en string(workplace_city eller occupation)
# annan funktion för att hämta ut en specifik stad och se data
# dashboard för att visa både dataframe och en plot på din df, hämta värde från en selectbox och skicka in till funktionen

,workplace_municipality,total vacancies
0,Göteborg,24878
1,Malmö,6906
2,Linköping,3641
3,Stockholm,3368
4,Örebro,2854


In [6]:
df_technical_occupation = group_by_occupation(df_technical)
df_education_occupation = group_by_occupation(df_education)
df_healthcare_occupation = group_by_occupation(df_healthcare)

df_technical_occupation.head()

,occupation,total vacancies
0,"Civilingenjör, maskin",634
1,"Civilingenjör, teknisk fysik",566
2,"Produktionstekniker, maskin/Produktionsteknike...",394
3,"Civilingenjör, konstruktion, elektronik",329
4,"Civilingenjör, elkraft",298


In [13]:
import streamlit as st
import matplotlib.pyplot as plt
import plotly.express as px

px.bar(
    df_city.head(15),
    x="workplace_municipality",
    y="total vacancies",
    color_discrete_sequence=px.colors.qualitative.Plotly,
)

In [8]:
import plotly.graph_objects as go 
from plotly.subplots import make_subplots

# gör en karta över everige och skapa färger baserat på occupation_field med flest vacancies för varje workplace_region, där man kan hovra över regionen och se mer info, typ vilka städer och vilka jobb inom fieldet som 

fig1 = px.pie(df_education, names="workplace_region",color_discrete_sequence=px.colors.qualitative.Plotly, values="vacancies", title="vacancies per region Education")
fig1.show()
fig2 = px.pie(df_healthcare, names="workplace_region",color_discrete_sequence=px.colors.qualitative.Plotly, values="vacancies", title="vacancies per region Healthcare")
fig2.show()
fig3 = px.pie(df_technical, names="workplace_region",color_discrete_sequence=px.colors.qualitative.Plotly, values="vacancies", title="vacancies per region Technical")
fig3.show()

# fig = make_subplots(rows=1, cols=3, subplot_titles=("Education", "Healthcare", "Technical"))
# for trace in fig1.data:
#     fig.add_trace(trace, row=1, col=1)
# for trace in fig2.data:
#     fig.add_trace(trace, row=1, col=2)
# for trace in fig3.data:
#     fig.add_trace(trace, row=1, col=3)
    
# fig.show()
